# The Battle of Neighborhoods: Boston, Massachusetts

Boston, Massachusetts is one of the most widely known cities in the United States. The city proper covers 127 km2 and, as of 2018, the estimated population is 694,583 (see [US Census Bureau](https://www.census.gov/quickfacts/fact/table/bostoncitymassachusetts/PST045218 "US Census Bureau")). It's not a very big city, which may even give an additional value to it: a tourist can explore city's many sides in a relatively short amount of time, and it could potentially be a very livable place.

### 1. Allston

There is a river (Charles River) and a bicycle path on the north and east, offering plenty of recreational opportunities. 

diverse community, including longtime residents, students from nearby Boston University and Harvard, recent immigrants, and young professionals
Supports business owners and nonprofits in the area. 
Harvard University, which owns more land in Boston than in Cambridge, is a major tenant of the neighborhood. 
Business school and athletic facilities.
64.55% of Allston’s population is between the ages of 20-34 making Allston the neighborhood with the highest percentage of young adults.
358 acres of land dedicated to academic space, the most in Boston. 


### 2. Back Bay 

protected historic district
Today Back Bay is known for its elegant architecture and bustling commercial streets
feature concerts and festivals
the finish line of the Boston Marathon is here
international retailers, salons, boutique offices, and restaurants
business
high-end hotels, restaurants, and office buildings
89.5% of Back Bay’s population has a bachelor’s degree or higher
The median age in Back Bay is 31.9

### 3. Bay Village 

One of the smallest neighborhoods in Boston
shops and restaurants
Bay Village's narrow streets, historic facades, brick sidewalks, and gas street lamps create an appealing and quiet pedestrian environment in a primarily residential neighborhood. Residents enjoy easy walking access to the surrounding parks, shops, restaurants, and theaters of Downtown, Back Bay, and the South End. 

### 4. Beacon Hill

protected historic neighborhood
likely Boston's best known neighborhood
Maasachussets State House
Classic red brick houses from the early nineteeth century
Eclectic boutiques, antique stores, and restaurants serve the neighborhood and Beacon Hill's frequent tourists. The Beacon Hill Business Association supports business owners and nonprofits in the area. 
89.5% of Beacon Hill’s population has a bachelor’s degree or higher. 
The median age in Beacon Hill is 31.3. 

### 5. Brighton

peaceful neighborhood is a great place for young families and professionals
residential and commercial
Families, who have called Brighton home for generations, share the neighborhood with recent immigrants, students from nearby colleges, and young professionals
This diversity of population attracts a diversity of businesses, nonprofits, medical and educational institutions to the area.
ethnic grocery stores to cozy neighborhood pubs. Major tenants of the neighborhood include St. Elizabeth's Medical Center, Boston College, WGBH, and the New Balance Athletic Shoe's world headquarters

### 6. Charlestown

neighborhood’s Irish roots 
The neighborhood has historical roots, but it has turned into a busy, modern-day neighborhood.
city's oldest neighborhood
public housing
waterfront condominiums and apartments
contemporary restaurants and shops thrive alongside the oldest tavern in Massachusetts (Warren Tavern)
two marinas
Bunker Hill Community College, Spauling Rehabilitation Hospital, and a branch of Massachusetts General Hospital

### 7. Chinatown-Leather District

Chinese quarter
water features and bamboo, Chinatown Park
South Station serves as a major rail and bus hub for Boston
The number of housing units in Chinatown grew 54.5% from 2000 to 2010.
49.2% of households in Chinatown are families

### 8. Dorchester

The biggest and most diverse neighborhood in the City
Long-time residents mingle together with new immigrants from Vietnam, Cape Verde, Ireland, and many other countries.
This wonderful mix of residents from all cultures and backgrounds makes it an incredibly vibrant place to live, work, and spend time. Former residents can often be seen and heard claiming to be “OFD” — Originally From Dorchester.
waterfront, residential neighborhoods, commercial corridors, and a university campus
numerous parks
University of Massachussets Boston

### 9. Downtown

Home to City Hall, numerous corporate headquarters, condos and apartments, and some of Boston’s most beloved tourist attractions 
in the heart of Boston
Housing options in downtown are varied, ranging from historic apartment buildings to modern glass towers. 
Over 200,000 people work downtown every day
major retail area, offering a range of apparel stores, national retailers, bakeries, and both fast casual and fine restaurants to suit every taste
Historic theaters, including the Opera House
hotels
Suffolk University, Emerson College, and Urban College

### 10.East Boston

waterfront
the neighborhood of immigrants
many ethnic restaurants
residential neighborhoods, transportation links, and recreational opportunities
East Boston is currently home to a blend of longtime residents, young professionals, and predominantly Colombian and Salvadoran immigrants
Logan International Airport
Almost 50% of East Boston’s population is foreign born, the highest percentage of all Boston’s neighborhoods. Over half of East Boston’s foreign born population is from Colombia or El Salvador.
family oriented: Almost 60% of East Boston’s households are families 

### 11. Fenway-Kenmore

Fenway Park and Red Sox
Museum of Fine Arts and Symphony Hall
clubs
higher education institutions
buzzing mix of families, young professionals, and students
Berklee College of Music
Huntington Avenue is celebrated as Boston’s “Avenue of the Arts”.
music venues and nightlife
numerous academic and medical institutions 
Almost 70% of Fenway’s population is enrolled in school. 
44.2% of Fenway’s population works in the educational services, health care and social assistance industry.


### 12. Hyde Park

suburban area
Today, Hyde Park offers its residents a unique blend of accessible city amenities and quiet suburban lifestyle
deiversed population
significant open and green space
A thriving industrial section of the neighborhood is home to numerous businesses.  

### 13. Jamaica Plain

diverse
suburb
home to Latinos, young families, a growing gay community, and young professionals
fishing, sailing, and running 
botanical garden
5% of Jamaica Plain’s population commutes to work by bicycle – the highest percentage of any Boston neighborhood. 
25.3% of Jamaica Plain’s population is Hispanic.

### 14. Mattapan 

home to diverse cultures and many immigrant-owned businesses
At the dawn of 21st century, immigrants traveling in New England found Mattapan to be a "good place to sit." Irish, Jewish, and Haitian immigrants called the neighborhood home. Now, Mattapan is home to a large African American and Caribbean community.
significant amount of green space
Blue Hill Avenue and Mattapan Square are the neighborhood’s main commercial districts, home to banks, law offices, restaurants, retail shops, health centers, and places of worship
67.3% of households in Mattapan are families.
Mattapan has the highest percentage of residents who speak French at home, at 18.4%. 

### 15. Mid-Dorchester

The City is bringing lots of new development to the area, including housing and light industrial and commercial opportunities. With the improvement of the Fairmount/ Indigo Line, there's now easy access to jobs and the downtown financial area.

### 16. Mission Hill

largely residential neighborhood 
traditional architecture
Mission Hill’s population grew 17% from 2000 to 2010. 
53% of Mission Hill residents are non-white


### 17. North End

Italian-American community
some of the City's best restaurants and old-world cafes
one of the most visited parts of Boston
Boston's oldest residential community 
Boston's "Little Italy"
historical sites
80.9% of North End’s population has a bachelor’s degree or higher. 
The North End experienced a 5.1% population increase from 2000 to 2010.  

### 18. Roslindale

garden suburb 
plenty natural beauty
primarely residential community
13.2% of Roslindale’s population is enrolled in nursery, kindergarten, elementary, or middle school. 
40.7% of Roslindale’s residents speak a language other than English at home.

### 19. Roxbury

historic neighborhood
Today, Roxbury is the heart of Black culture in Boston
geographic heart of Boston
Hispanic, and Asian families, along with young professionals
parks, substantial green space
Roxbury Community College and the John D. O'Bryant School of Mathematics and Science 
Roxbury is home to over 570 acres of parks, playgrounds, and athletic fields, the most in the city.  Roxbury has over 23% of park space in Boston. 
Roxbury’s population grew by 16.8% from 2000 to 2010.  

### 20. South Boston

An urban neighborhood with a strong sense of history and tradition
South Boston is home to both long-time residents and a new wave of young professionals who are drawn to the area’s open space, emerging nightlife, and easy access to downtown. The neighborhood boasts miles of beaches and waterfront parks
Between 2000 and 2010, South Boston’s population with a bachelor’s degree or higher increased from 28.2% to 54.8%.
South Boston’s Black or African American population grew by 47.9% from 2000 to 2010. 

### 21. South End 

The South End is truly a cosmopolitan neighborhood. It is just minutes away from Downtown and the Back Bay, and has recently become one of the City’s most popular neighborhoods.
Victorian townhouses
home to active young families, professionals, and immigrants, and is popular with Boston’s gay community
United States' largest Victorian residential district
Boston Center for the Arts
Boston Medical Center and Boston University School of Medicine
44.7% of South End residents are non-white. 
11.2% of South End’s population speaks Chinese at home

### 22. West End

home to prominent Boston institutions, including Massachusetts General Hospital and TD Garden
up-and-coming neighborhood in Boston
large apartment towers
small neighborhood
City of Cambridge - walking distance
The Liberty Hotel and Clink Bar, both housed in a converted historic jail, are hip neighborhood institutions
93.4% of West End’s population has a bachelor’s degree or higher, the highest percentage of all Boston neighborhoods. 
32.2% of West End’s population is foreign born. 

### 23. West Roxbury 

civic activism and youth programs
suburban vibe
green spaces
swimming
hiking
West Roxbury has the most open space in the city, with nearly 1,200 acres. 


In [1]:
import numpy as np 
import pandas as pd
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Everything works fine!')

Everything works fine!


In [2]:
res = requests.get(
    'https://www.boston.gov/neighborhoods')

soup = BeautifulSoup(res.text, 'html.parser')

boston_neighborhoods = []

for row in soup.find_all('div', class_='g p-b500')[0].find_all('h3'):
    boston_neighborhoods.append(row.text)
print(boston_neighborhoods)


['Allston', 'Back Bay', 'Bay Village', 'Beacon Hill', 'Brighton', 'Charlestown', 'Chinatown-Leather District', 'Dorchester', 'Downtown', 'East Boston', 'Fenway-Kenmore', 'Hyde Park', 'Jamaica Plain', 'Mattapan', 'Mid-Dorchester', 'Mission Hill', 'North End', 'Roslindale', 'Roxbury', 'South Boston', 'South End', 'West End', 'West Roxbury']


In [3]:
df = pd.DataFrame({'Boston Neighborhoods': boston_neighborhoods})
df

,Boston Neighborhoods
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton
5,Charlestown
6,Chinatown-Leather District
7,Dorchester
8,Downtown
9,East Boston


In [37]:
df.shape[0]

23

In [4]:
def find_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Boston, Massachusetts'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
lat_long = [find_latlng(neighborhood) for neighborhood in df['Boston Neighborhoods']]

In [6]:
lat_long

[[42.35046649714098, -71.11100415506229],
 [42.34999000000005, -71.08764999999994],
 [42.348168000000015, -71.06847074999999],
 [42.35842000000008, -71.06859999999995],
 [42.35213123053123, -71.12492353661709],
 [42.36776992509806, -71.05901694825978],
 [42.35251000000005, -71.06089999999995],
 [42.35134867454975, -71.05285041243937],
 [42.35829000000007, -71.05662999999998],
 [42.35141360528571, -71.05671282961363],
 [42.34355000000005, -71.10156999999998],
 [42.274893442956426, -71.1199472126674],
 [42.30584990221593, -71.11909514176067],
 [42.278224926536424, -71.09608297802978],
 [42.35134867454975, -71.05285041243937],
 [42.33578000000006, -71.10980999999998],
 [42.36549000000008, -71.05296999999996],
 [42.28181311006276, -71.13711121314374],
 [42.33029675765322, -71.0894809461078],
 [42.35224995700159, -71.0556905214992],
 [42.34256000000005, -71.07357999999994],
 [42.36394000000007, -71.06738999999999],
 [42.28219344914676, -71.14599477055154]]

In [7]:
df_coordinates = pd.DataFrame(lat_long, columns=('Latitude', 'Longitude'))
df_coordinates

,Latitude,Longitude
0,42.350466,-71.111004
1,42.349990,-71.087650
2,42.348168,-71.068471
3,42.358420,-71.068600
4,42.352131,-71.124924
5,42.367770,-71.059017
6,42.352510,-71.060900
7,42.351349,-71.052850
8,42.358290,-71.056630
9,42.351414,-71.056713


In [8]:
df = df.assign(Latitude=df_coordinates.Latitude.values, Longitude=df_coordinates.Longitude.values)
df

,Boston Neighborhoods,Latitude,Longitude
0,Allston,42.350466,-71.111004
1,Back Bay,42.349990,-71.087650
2,Bay Village,42.348168,-71.068471
3,Beacon Hill,42.358420,-71.068600
4,Brighton,42.352131,-71.124924
5,Charlestown,42.367770,-71.059017
6,Chinatown-Leather District,42.352510,-71.060900
7,Dorchester,42.351349,-71.052850
8,Downtown,42.358290,-71.056630
9,East Boston,42.351414,-71.056713


In [24]:
df['Zip Code'] = ['02134', '02116', '02116', '02108', '02135', '02129', '02111', '02121', '02201', '02128', '[02115, 02215]', '02136', '02130', '02126', '[02121, 02122, 02124, 02125]', '[02120, 02115]', '02113', '02131', '02119', '02127', '02118', '02114', '02132']

In [25]:
df['Zip Code']
df

,Boston Neighborhoods,Latitude,Longitude,Zip Code
0,Allston,42.350466,-71.111004,02134
1,Back Bay,42.349990,-71.087650,02116
2,Bay Village,42.348168,-71.068471,02116
3,Beacon Hill,42.358420,-71.068600,02108
4,Brighton,42.352131,-71.124924,02135
5,Charlestown,42.367770,-71.059017,02129
6,Chinatown-Leather District,42.352510,-71.060900,02111
7,Dorchester,42.351349,-71.052850,02121
8,Downtown,42.358290,-71.056630,02201
9,East Boston,42.351414,-71.056713,02128


In [27]:
df['Zip Code'] = df['Zip Code'].str.strip('[]')
df

,Boston Neighborhoods,Latitude,Longitude,Zip Code
0,Allston,42.350466,-71.111004,02134
1,Back Bay,42.349990,-71.087650,02116
2,Bay Village,42.348168,-71.068471,02116
3,Beacon Hill,42.358420,-71.068600,02108
4,Brighton,42.352131,-71.124924,02135
5,Charlestown,42.367770,-71.059017,02129
6,Chinatown-Leather District,42.352510,-71.060900,02111
7,Dorchester,42.351349,-71.052850,02121
8,Downtown,42.358290,-71.056630,02201
9,East Boston,42.351414,-71.056713,02128


In [10]:
address = 'Boston, Massachusetts'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of '+ address + ' are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Boston, Massachusetts are 42.3602534, -71.0582912.


In [11]:
map_boston = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Boston Neighborhoods']):
    label = '{}'.format(neighborhood)
    folium.Marker([lat, lng], popup=label).add_to(map_boston) 
    
map_boston